In [11]:
import pandas as pd

In [12]:
#Carregando arquivo com dados do IPCA
FILE = 'IPCA-serie-historica.csv/mes_brasil.csv'

In [13]:
df = pd.read_csv('../getters/Data/extracted/'+FILE)

In [15]:
df

,ano,mes,indice,variacao_mensal,variacao_trimestral,variacao_semestral,variacao_anual,variacao_doze_meses
0,1979,12,7.618300e-09,NaN,NaN,NaN,NaN,NaN
1,1980,1,8.122300e-09,6.62,NaN,NaN,6.62,NaN
2,1980,2,8.497300e-09,4.62,NaN,NaN,11.54,NaN
3,1980,3,9.010400e-09,6.04,18.27,NaN,18.27,NaN
4,1980,4,9.486700e-09,5.29,16.80,NaN,24.53,NaN
...,...,...,...,...,...,...,...,...
500,2021,8,5.876050e+03,0.87,2.38,4.51,5.67,9.68
501,2021,9,5.944210e+03,1.16,3.02,4.75,6.90,10.25
502,2021,10,6.018510e+03,1.25,3.32,5.73,8.24,10.67
503,2021,11,6.075690e+03,0.95,3.40,5.86,9.26,10.74
